## Set up script

In [ ]:
import os
from netCDF4 import Dataset
import numpy as npy
import numpy as np ## messy coding, sorry - i use npy but everyone else uses np...
from math import *
import matplotlib.pylab as plt
import xarray as xr
import sys
#!conda install --yes --prefix {sys.prefix} cv2
!{sys.executable} -m pip install ffmpeg
import ffmpeg

########################
figdir = '/Users/heareg/Documents/Floes/working_directory/figures/'
    
use_OSISAF = 0
use_Bremen = 1

if use_OSISAF:
    
    datadir = '/Users/heareg/Documents/OSISAF_SH/'
    fname_conc_st = 'ice_conc_sh_polstere-100_multi_'
    OSI_ice_file = datadir+'2022/07/'+fname_conc_st+'202207221200.nc'
    fname_type_st = 'ice_type_sh_polstere-100_multi_'
    OSI_type_file = datadir+'2022_type/07/'+fname_type_st+'202207221200.nc'
    figdir = figdir + 'OSI'
    
elif use_Bremen:
    
    #rawdatadir = '/Users/heareg/Documents/IceType_SH_Bremen/'
    datadir = '/Users/heareg/Documents/Floes/working_directory/floes/contributors/Heather/data/for_iceconctype/'
    typedir = datadir+'RAW/'
    fname_type_st = 'ECICE-IcetypesUncorrected-'
    BREMEN_type_file = typedir+'2022/'+fname_type_st+'20220722.nc'
    BREMEN_cell_area = datadir+'GridCellArea/PolStereo_GridCellArea_s12.5km_Antarctic.nc'
    figdir = figdir + 'Bremen'
    
else: 
    
    print('ERROR!')
    sys.exit()
    
ice_conc_ref = datadir+'ice_conc_sh_polstere-100_multi_202001171200.nc'
ATL20_thickness_file = datadir+'ATL20.nc'

########################

def findfile(thisfname,return_status=0):
    this_status = os.path.isfile(thisfname)
    if this_status:
        print('---- FOUND ---- '+thisfname)
    else:
        print('---- NOT FOUND ---- '+thisfname)
    if return_status:
        return this_status
        
print('THINGS TO DO BEFORE PROCEEDING:')
print('1) check that '+figdir+' exists')
print('2) check that '+datadir+' exists and it contains the following files:')
if use_Bremen:
    findfile(BREMEN_type_file)
    findfile(BREMEN_cell_area)
else:
    findfile(OSI_ice_file)
    findfile(OSI_type_file)
findfile(ice_conc_ref)
findfile(ATL20_thickness_file)
fexist = findfile(datadir+'BremenMYIfrac_ts_2019-2022_out_of_full_2019-2022.npz',return_status=1)
if fexist:
    print('...will be loading MYI fraction timeseries in')
else:
    print('...will need to compute MYI fraction timeseries')

## Define regions for OSISAF or Bremen data

In [ ]:
if use_OSISAF:
    
    #datadir = '/Users/heareg/Documents/OSISAF_SH/'
    this_f = Dataset(OSI_ice_file)
    this_f.variables
    lat = npy.squeeze(npy.array(this_f.variables['lat']))
    lon = npy.squeeze(npy.array(this_f.variables['lon']))
    ice_conc = npy.squeeze(npy.array(this_f.variables['ice_conc']))
    status_flag = npy.squeeze(npy.array(this_f.variables['status_flag']))
    
    this_t = Dataset(OSI_type_file)
    ice_type = npy.squeeze(npy.array(this_t.variables['ice_type']))
    ### 1 = no/little ice, 2 = young ice, 3 = myi, 4 = ambiguous 
    
    WWSbox = npy.zeros(npy.shape(lon))
    WWSbox[(lon>-65)&(lon<-45)&(lat>-77.6)&(lat<-62)] = 1
    SWSbox = npy.zeros(npy.shape(lon))
    SWSbox[(lon>-65)&(lon<-15)&(lat>-77.6)&(lat<-73)] = 1
    Bothbox = npy.zeros(npy.shape(lon))
    Bothbox[WWSbox+SWSbox==2] = 1
    
    ### So WWS box is -65 < lon < -45, -73 < lat < -62
    ### SWS box is -65 < lon < 15, -77.6 < lat < -73

    landmask = npy.ones(npy.shape(ice_conc))
    landmask[status_flag>=100] = npy.nan
    ice_conc[status_flag>=100] = npy.nan
    
    cell_area = 12500 ## ish

elif use_Bremen:

    #datadir = '/Users/heareg/Documents/IceType_SH_Bremen/'
    this_t = Dataset(BREMEN_type_file)    
    OW = npy.squeeze(npy.array(this_t.variables['OW']))
    YI = npy.squeeze(npy.array(this_t.variables['YI']))
    FYI = npy.squeeze(npy.array(this_t.variables['FYI']))
    MYI = npy.squeeze(npy.array(this_t.variables['MYI']))
    lon = npy.squeeze(npy.array(this_t.variables['LON']))
    lat = npy.squeeze(npy.array(this_t.variables['LAT']))
    
    ice_conc = YI + FYI + MYI;
    
    WWSbox = npy.zeros(npy.shape(lon))
    WWSbox[(lon>-65)&(lon<-45)&(lat>-77.6)&(lat<-62)] = 1
    SWSbox = npy.zeros(npy.shape(lon))
    SWSbox[(lon>-65)&(lon<-15)&(lat>-77.6)&(lat<-73)] = 1
    Bothbox = npy.zeros(npy.shape(lon))
    Bothbox[WWSbox+SWSbox==2] = 1

    landmask = npy.ones(npy.shape(ice_conc))
    landmask_10 = npy.zeros(npy.shape(ice_conc))
    landmask[npy.isnan(ice_conc)] = npy.nan
    landmask_10[npy.isnan(ice_conc)] = 1
    ice_conc[npy.isnan(ice_conc)] = npy.nan
    
    cell_area = npy.squeeze(npy.array(Dataset(BREMEN_cell_area).variables['data']))
    print('here')

plt.figure()
plt.pcolormesh(npy.flipud(ice_conc*landmask),vmin=0,vmax=1,cmap='Blues_r'); plt.colorbar()
plt.contour(npy.flipud(WWSbox),colors='r')
plt.contour(npy.flipud(SWSbox),colors='orange')
plt.contour(npy.flipud(Bothbox),colors='blue')
plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
if use_OSISAF:
    plt.contour(npy.flipud(status_flag),[100,101],colors='k')
    plt.xlim([100,400])
    plt.ylim([400,700])
else:
    plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
    plt.xlim([50,320])
    plt.ylim([300,570])
    
plt.figure()
plt.contourf(npy.flipud(MYI*landmask),4,cmap='Spectral_r'); plt.colorbar()
plt.contour(npy.flipud(WWSbox),colors='r')
plt.contour(npy.flipud(SWSbox),colors='orange')
plt.contour(npy.flipud(Bothbox),colors='blue')
if use_OSISAF:
    plt.contour(npy.flipud(status_flag),[100,101],colors='k')
    plt.xlim([100,400])
    plt.ylim([400,700])
else:
    plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
    plt.xlim([50,320])
    plt.ylim([300,570])

plt.figure()
plt.contourf(npy.flipud(MYI),4,cmap='Spectral_r'); plt.colorbar()
plt.xlim([50,320])
plt.ylim([300,570])


def dist(lon1,lat1,lon2,lat2):

  lon1_rad = np.radians(lon1)
  lon2_rad = np.radians(lon2)
  lat1_rad = np.radians(lat1)
  lat2_rad = np.radians(lat2)
  #Assumes degrees input
  #Calculates in metres
  R = 6371000 #Radius of earth in metres (roughly)
  ## Uses Haversine formula
  a1 = (sin((lat2_rad-lat1_rad)/2))**2
  a2 = (cos(lat1_rad))*(cos(lat2_rad))*((sin((lon2_rad-lon1_rad)/2))**2)
  a = a1 + a2
  c = 2*atan2(sqrt(a),sqrt(1-a))
  d = R*c

  return d

def area(lon_in,lat_in):

  ## At the moment it assumes lat and lon are 1d. If 2d, need to do something about populating dx and dy
  if npy.ndim(lon_in) == 1:
    lon = lon_in.copy()
    ## If it's 1d
    londim = lon.size
    lon_start = lon[0]-(lon[1]-lon[0])/2
    lon_end = lon[-1]+(lon[-1]-lon[-2])/2
    lon_av = npy.array((lon[-(londim-1):]+lon[0:londim-1])/2)#lon_av = npy.array((lon[1:-1]+lon[0:-2])/2)
    lon_loop = npy.concatenate(([lon_start],lon_av,[lon_end]))
  else:
    #Do this bit
    londim = lon_in.shape[1]
    lon_loop = lon_in*1

  if npy.ndim(lat_in) == 1:
    lat = lat_in.copy()
    latdim = lat.size
    lat_start = lat[0]-(lat[1]-lat[0])/2
    lat_end = lat[-1]+(lat[-1]-lat[-2])/2
    lat_av = (lat[-(latdim-1):]+lat[0:latdim-1])/2
    lat_loop = npy.concatenate(([lat_start],lat_av,[lat_end]))
  else:
    #Do this bit
    latdim = lat_in.shape[0]
    lat_loop = lat_in*1

  dx = -999*npy.ones([latdim,londim])
  dy = -999*npy.ones([latdim,londim])

  #Loop over longitudes to populate latitude distances
  for loncnt in range(0,londim):
    for lat_inc in range(0,latdim):
      if npy.ndim(lon_in) == 1:
        dy[lat_inc,loncnt] = npy.abs(dist(lon_in[loncnt],lat_loop[lat_inc],lon_in[loncnt],lat_loop[lat_inc+1]))
      else:
        if lat_inc < latdim-1:
          dy[lat_inc,loncnt] = npy.abs(dist(lon_in[lat_inc,loncnt],lat_in[lat_inc,loncnt],lon_in[lat_inc,loncnt],lat_in[lat_inc+1,loncnt]))
#      dy[lat_inc,loncnt] = CalculateArea.dist(float(lon[loncnt]),float(lat_loop[lat_inc]),float(lon[loncnt]),float(lat_loop[lat_inc+1]))


  for latcnt in range(0,latdim):
    for lon_inc in range(0,londim):
      if npy.ndim(lat_in) == 1:
        dx[latcnt,lon_inc] = npy.abs(dist(lon_loop[lon_inc],lat_in[latcnt],lon_loop[lon_inc+1],lat_in[latcnt]))
      else:
        if lat_inc < londim-1:
          dx[latcnt,lon_inc] = npy.abs(dist(lon_in[latcnt,lon_inc],lat_in[latcnt,lon_inc],lon_in[latcnt,lon_inc+1],lat_in[latcnt,lon_inc]))
#      dx[latcnt,lon_inc] = CalculateArea.dist(lon_loop[lon_inc],lat[latcnt],lon_loop[lon_inc+1],lat[latcnt])

  return dx, dy

if use_OSISAF:
    dx,dy = area(lon,lat)
    grid_area = dx*dy
else:
    grid_area = cell_area

## Get monthly sea ice thickness

In [ ]:
#### Find a better way to do this!!! Bit hacky with date...

ATL20_lat = npy.squeeze(npy.array(Dataset(ice_conc_ref).variables['lat']))
ATL20_lon = npy.squeeze(npy.array(Dataset(ice_conc_ref).variables['lon']))

ATL20_west_in = npy.zeros(npy.shape(ATL20_lat))
ATL20_west_in[(ATL20_lon>-65)&(ATL20_lon<-45)&(ATL20_lat>-77.6)&(ATL20_lat<-62)] = 1
ATL20_south_in = npy.zeros(npy.shape(ATL20_lat))
ATL20_south_in[(ATL20_lon>-65)&(ATL20_lon<-15)&(ATL20_lat>-77.6)&(ATL20_lat<-73)] = 1
ATL20_west_in[ATL20_south_in==1] = 0
print(npy.shape(ATL20_west_in))

ATL20_thick_monthly = npy.squeeze(npy.array(Dataset(datadir+'ATL20.nc').variables['mean_fb']))

lat_shp = npy.shape(ATL20_lat)
ATL20_west_in = ATL20_west_in[83:lat_shp[0]-83,79:lat_shp[1]-79]*1
ATL20_south_in = ATL20_south_in[83:lat_shp[0]-83,79:lat_shp[1]-79]*1

ATL20_west = ATL20_west_in[1:lat_shp[0]:2,1:lat_shp[1]:2]*1
ATL20_south = ATL20_south_in[1:lat_shp[0]:2,1:lat_shp[1]:2]*1
ATL20_west_nan = ATL20_west*1; ATL20_west_nan[ATL20_west==0] = npy.nan
ATL20_south_nan = ATL20_south*1; ATL20_south_nan[ATL20_south==0] = npy.nan
print(npy.shape(ATL20_west))

### From Mukund and Sean:
ds = xr.open_dataset(ATL20_thickness_file)
fb_south = ds.where((ds['grid_x'] > -1225655) & (ds['grid_x'] < -481459) & (ds['grid_y'] >  571532) & (ds['grid_y'] < 1796829))['mean_fb'].mean(dim=['grid_x', 'grid_y'])
fb_west = ds.where((ds['grid_x'] < -1225655) & (ds['grid_x'] > -2193339) & (ds['grid_y'] >  571532) & (ds['grid_y'] < 2193339))['mean_fb'].mean(dim=['grid_x', 'grid_y'])
grid_x = ds['grid_x'].values
grid_y = ds['grid_y'].values
print(npy.shape(grid_x))
south_grid = npy.ones(npy.shape(grid_x))

print(npy.shape(fb_south))
print(npy.shape(ATL20_south))
plt.figure()
plt.pcolormesh(ATL20_lat); plt.colorbar()
plt.figure()
plt.pcolormesh(ATL20_thick_monthly[6,:,:],vmin=-0.5,vmax=0.5)
plt.contour(ATL20_west)
plt.contour(ATL20_south)

ninc = 0.05;
nmax = 3
ntest = int(nmax/ninc); print(ntest)

#ATL20_thick_daily_west = npy.zeros([npy.shape(ATL20_thick_daily)[0],ntest])
#ATL20_thick_daily_south = npy.zeros([npy.shape(ATL20_thick_daily)[0],ntest])
ATL20_thick_monthly_west = npy.zeros([npy.shape(ATL20_thick_monthly)[0],ntest])
ATL20_thick_monthly_south = npy.zeros([npy.shape(ATL20_thick_monthly)[0],ntest])

#ATL20_thick_daily_west_av = npy.zeros([npy.shape(ATL20_thick_daily)[0],3])
#ATL20_thick_daily_south_av = npy.zeros([npy.shape(ATL20_thick_daily)[0],3])
ATL20_thick_monthly_west_av = npy.zeros([npy.shape(ATL20_thick_monthly)[0],3])
ATL20_thick_monthly_south_av = npy.zeros([npy.shape(ATL20_thick_monthly)[0],3])


daily_time = npy.arange(2018+(31+28+31+30+31+30+31+31+30+1)/365,2022+46/365,1./365)
        
for nn in range(0,npy.shape(ATL20_thick_monthly)[0]):
    
    this_thick = ATL20_thick_monthly[nn,:,:]*1
    this_thick[this_thick>100] = npy.nan
    
    this_thick_west = this_thick*ATL20_west_nan
    this_thick_south = this_thick*ATL20_south_nan
    
    this_thick_west[this_thick_west<=0] = npy.nan
    this_thick_south[this_thick_south<=0] = npy.nan
    
    ATL20_thick_monthly_west_av[nn,0] = npy.nanmin(this_thick_west)
    ATL20_thick_monthly_west_av[nn,1] = npy.nanmean(this_thick_west)
    ATL20_thick_monthly_west_av[nn,2] = npy.nanmax(this_thick_west)
    ATL20_thick_monthly_south_av[nn,0] = npy.nanmin(this_thick_south)
    ATL20_thick_monthly_south_av[nn,1] = npy.nanmean(this_thick_south)
    ATL20_thick_monthly_south_av[nn,2] = npy.nanmax(this_thick_south)

    ## eventually need to weight by area of each grid cell
    maxcnt = ninc
    thisbin = 0
    for xn in npy.arange(0,nmax,ninc):
        
        westvals = this_thick_west*1
        westvals[westvals<xn] = npy.nan
        westvals[westvals>xn+ninc] = npy.nan
        ind = npy.nonzero(npy.isnan(westvals)==0)[0]
        ATL20_thick_monthly_west[nn,thisbin] = len(ind)#npy.nansum(westvals)
        
        southvals = this_thick_south*1
        southvals[southvals<xn] = npy.nan
        southvals[southvals>xn+ninc] = npy.nan
        ind = npy.nonzero(npy.isnan(southvals)==0)[0]
        ATL20_thick_monthly_south[nn,thisbin] = len(ind)#npy.nansum(southvals)
        
        maxcnt = maxcnt + ninc
        thisbin = thisbin + 1
    
    if npy.mod(nn,3) == 0:
        print(str(nn)+' of '+str(npy.shape(ATL20_thick_monthly)[0]))


## Now we want to obtain timeseries of ice type

### OSISAF option

In [ ]:
def get_conc_and_type_OSISAF(year,month,day):
        datadir = '/Users/heareg/Documents/OSISAF_SH/'
        fname_conc_st = 'ice_conc_sh_polstere-100_multi_'
        fname_type_st = 'ice_type_sh_polstere-100_multi_'

        ice_conc = npy.nan
        ice_type = npy.nan
        landmask = npy.nan
        lat = npy.nan
        lon = npy.nan


        exist = 1

        fn = datadir+str(year)+'/'+str(month).zfill(2)+'/'+fname_conc_st+str(year)+str(month).zfill(2)+str(day).zfill(2)+'1200.nc'
        if os.path.isfile(fn):
            this_f = Dataset(fn)
            lat = npy.squeeze(npy.array(this_f.variables['lat']))
            lon = npy.squeeze(npy.array(this_f.variables['lon']))
            ice_conc = npy.squeeze(npy.array(this_f.variables['ice_conc']))/100
            status_flag = npy.squeeze(npy.array(this_f.variables['status_flag']))

            landmask = npy.ones(npy.shape(ice_conc))
            landmask[status_flag>=100] = npy.nan
            ice_conc[status_flag>=100] = npy.nan
        else:
            print(fn+' does not exist')

        tn = datadir+str(year)+'_type/'+str(month).zfill(2)+'/'+fname_type_st+str(year)+str(month).zfill(2)+str(day).zfill(2)+'1200.nc'
        if os.path.isfile(tn):
            this_t = Dataset(tn)
            ice_type = npy.squeeze(npy.array(this_t.variables['ice_type']))
            ### 1 = no/little ice, 2 = young ice, 3 = myi, 4 = ambiguous 

            if os.path.isfile(fn) == 0:
                landmask = npy.ones(npy.shape(ice_type))
                landmask[ice_type==-1] = npy.nan
        else:
            print(tn+' does not exist')
            exist = 0

        return lat,lon,ice_conc*landmask,ice_type*landmask,exist

if use_OSISAF:
    ## Read in some of the POLSTERE variables and compute amounts in each cell
    plot_figs = 1
    plt.rcParams['axes.facecolor'] = 'white'


    yrange =range(2021,2023)
    si_area = npy.nan*npy.ones([len(yrange)*366,5])
    myi_area = npy.nan*npy.ones([len(yrange)*366,5])
    fyi_area = npy.nan*npy.ones([len(yrange)*366,5])
    amb_area = npy.nan*npy.ones([len(yrange)*366,5])
    owt_area = npy.nan*npy.ones([len(yrange)*366,5])

    yax = npy.nan*npy.ones([len(yrange)*366,1])
    m_points = npy.nan*npy.ones([len(yrange)*12,1])

    cnt = 0
    mcnt = 0

    grid_area = 5*5/1e6

    lat,lon,sic,typ,exist = get_conc_and_type_OSISAF(2019,12,30)

    WWS_area = npy.nansum(npy.nansum(grid_area*WWSbox*landmask,axis=1),axis=0)
    SWS_area = npy.nansum(npy.nansum(grid_area*SWSbox*landmask,axis=1),axis=0)
    Both_area = npy.nansum(npy.nansum(grid_area*Bothbox*landmask,axis=1),axis=0)


    WWSnocbox = WWSbox*1
    WWSnocbox[WWSbox+SWSbox==2] = 0
    SWSnocbox = SWSbox*1
    SWSnocbox[WWSbox+SWSbox==2] = 0

    WWS_nocorner_area = npy.nansum(npy.nansum(grid_area*WWSnocbox*landmask,axis=1),axis=0)
    SWS_nocorner_area = npy.nansum(npy.nansum(grid_area*SWSnocbox*landmask,axis=1),axis=0)

    for yr in yrange:
        mst = 0
        if npy.mod(yr,1972) == 0:
            mlenfeb = 29
            ylen = 366
        else:
            mlenfeb = 28
            ylen = 365
        for m in range(1,13):
            mlen = 31
            if m in [4,6,9,11]:
                mlen = 30
            elif m == 2:
                mlen = mlenfeb
            for d in range(1,mlen):

                yax[cnt] = yr + (mst + d)/ylen

                lat,lon,sic,typ,exist = get_conc_and_type_OSISAF(yr,m,d)

                si_area[cnt,0] = npy.nansum(npy.nansum(sic*grid_area*WWSbox,axis=1),axis=0)/WWS_area
                si_area[cnt,1] = npy.nansum(npy.nansum(sic*grid_area*SWSbox,axis=1),axis=0)/SWS_area
                si_area[cnt,2] = npy.nansum(npy.nansum(sic*grid_area*Bothbox,axis=1),axis=0)/Both_area
                si_area[cnt,3] = npy.nansum(npy.nansum(sic*grid_area*WWSnocbox,axis=1),axis=0)/WWS_nocorner_area
                si_area[cnt,4] = npy.nansum(npy.nansum(sic*grid_area*SWSnocbox,axis=1),axis=0)/SWS_nocorner_area

                if exist == 1:
                    myivals = npy.zeros(npy.shape(typ)); myivals[(typ==3.)] = 1;
                    #print(npy.nansum(myivals))
                    myi_area[cnt,0] = npy.nansum(npy.nansum(myivals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                    myi_area[cnt,1] = npy.nansum(npy.nansum(myivals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                    myi_area[cnt,2] = npy.nansum(npy.nansum(myivals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                    myi_area[cnt,3] = npy.nansum(npy.nansum(myivals*grid_area*WWSnocbox,axis=1),axis=0)#/SWS_area
                    myi_area[cnt,4] = npy.nansum(npy.nansum(myivals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                    fyivals = npy.zeros(npy.shape(typ)); fyivals[typ==2] = 1
                    #print(npy.nansum(fyivals))
                    fyi_area[cnt,0] = npy.nansum(npy.nansum(fyivals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                    fyi_area[cnt,1] = npy.nansum(npy.nansum(fyivals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                    fyi_area[cnt,2] = npy.nansum(npy.nansum(fyivals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                    fyi_area[cnt,3] = npy.nansum(npy.nansum(fyivals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                    fyi_area[cnt,4] = npy.nansum(npy.nansum(fyivals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                    ambvals = npy.zeros(npy.shape(typ)); ambvals[typ==4] = 1
                    #print(npy.nansum(ambvals))
                    amb_area[cnt,0] = npy.nansum(npy.nansum(ambvals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                    amb_area[cnt,1] = npy.nansum(npy.nansum(ambvals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                    amb_area[cnt,2] = npy.nansum(npy.nansum(ambvals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                    amb_area[cnt,3] = npy.nansum(npy.nansum(ambvals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                    amb_area[cnt,4] = npy.nansum(npy.nansum(ambvals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                    owtvals = npy.zeros(npy.shape(typ));  owtvals[typ==1] = 1
                    #print(npy.nansum(owtvals))
                    owt_area[cnt,0] = npy.nansum(npy.nansum(owtvals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                    owt_area[cnt,1] = npy.nansum(npy.nansum(owtvals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                    owt_area[cnt,2] = npy.nansum(npy.nansum(owtvals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                    owt_area[cnt,3] = npy.nansum(npy.nansum(owtvals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                    owt_area[cnt,4] = npy.nansum(npy.nansum(owtvals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                    if plot_figs:
                        plt.figure()
                        plt.pcolormesh(npy.flipud(sic*landmask),vmin=0,vmax=1,cmap='Blues_r'); plt.colorbar()
                        plt.contour(npy.flipud(WWSbox),colors='r')
                        plt.contour(npy.flipud(SWSbox),colors='g')
                        plt.contour(npy.flipud(Bothbox),colors='blue')
                        plt.contour(npy.flipud(status_flag),[100,101],colors='k')
                        if use_OSISAF:
                            plt.contour(npy.flipud(status_flag),[100,101],colors='k')
                            plt.xlim([100,400])
                            plt.ylim([400,700])
                        else:
                            plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
                            plt.xlim([50,320])
                            plt.ylim([300,570])
                        plt.title('SIC_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2))
                        plt.savefig(figdir+'SICmaps/SIC_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2)+'.png',bbox_inches='tight')
                        plt.close()

                        plt.figure()
                        plt.contourf(npy.flipud(typ*landmask),4,cmap='Spectral'); plt.colorbar()
                        plt.contour(npy.flipud(WWSbox),colors='r')
                        plt.contour(npy.flipud(SWSbox),colors='g')
                        plt.contour(npy.flipud(Bothbox),colors='blue')
                        plt.contour(npy.flipud(status_flag),[100,101],colors='k')          
                        if use_OSISAF:
                            plt.contour(npy.flipud(status_flag),[100,101],colors='k')
                            plt.xlim([100,400])
                            plt.ylim([400,700])
                        else:
                            plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
                            plt.xlim([50,320])
                            plt.ylim([300,570])
                        plt.title('IceType_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2))
                        plt.savefig(figdir+'IceTypemaps/IceType_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2)+'.png',bbox_inches='tight')
                        plt.close()

                cnt = cnt + 1

            print(str(yr)+'m'+str(m).zfill(2))
            mst = mst + mlen 
            m_points[mcnt,0] = yr + mst/ylen
            mcnt = mcnt + 1

### Bremen option

In [ ]:
def get_conc_and_type(year,month,day):
        
        
        #datadir = '/Users/heareg/Documents/IceType_SH_Bremen/'
        #typedir = datadir+'RAW/'
        #fname_type_st = 'ECICE-IcetypesUncorrected-'
          
        ice_conc = npy.nan
        ice_type = npy.nan
        landmask = npy.nan
        lat = npy.nan
        lon = npy.nan
        MYI = npy.nan
        FYI = npy.nan
        YI = npy.nan
        OW = npy.nan

        exist = 1

        fn = typedir+str(year)+'/'+fname_type_st+str(year)+str(month).zfill(2)+str(day).zfill(2)+'.nc'
        if os.path.isfile(fn):
            this_t = Dataset(fn)    
            OW = npy.squeeze(npy.array(this_t.variables['OW']))
            YI = npy.squeeze(npy.array(this_t.variables['YI']))
            FYI = npy.squeeze(npy.array(this_t.variables['FYI']))
            MYI = npy.squeeze(npy.array(this_t.variables['MYI']))
            lon = npy.squeeze(npy.array(this_t.variables['LON']))
            lat = npy.squeeze(npy.array(this_t.variables['LAT']))

            ice_conc = YI + FYI + MYI;

            WWSbox = npy.zeros(npy.shape(lon))
            WWSbox[(lon>-65)&(lon<-45)&(lat>-77.6)&(lat<-62)] = 1
            SWSbox = npy.zeros(npy.shape(lon))
            SWSbox[(lon>-65)&(lon<-15)&(lat>-77.6)&(lat<-73)] = 1
            Bothbox = npy.zeros(npy.shape(lon))
            Bothbox[WWSbox+SWSbox==2] = 1

            landmask = npy.ones(npy.shape(ice_conc))
            landmask_10 = npy.zeros(npy.shape(ice_conc))
            landmask[npy.isnan(ice_conc)] = npy.nan
            landmask_10[npy.isnan(ice_conc)] = 1
            ice_conc[npy.isnan(ice_conc)] = npy.nan
            
        else:
            print(fn+' does not exist')
            exist = 0

        return lat,lon,ice_conc*landmask,MYI,FYI,YI,OW,exist

if use_Bremen:
    
  plot_figs = 0; only_when_not_exist = 1
  compute_vals = 0
    
  sy_full = 2019
  ey_full = 2022
  sy = 2019
  ey = 2022
  savename = 'BremenMYIfrac_ts_'+str(sy)+'-'+str(ey)+'_out_of_full_'+str(sy_full)+'-'+str(ey_full)+'.npz'

  if sy == sy_full:
        cnt = 0
  elif sy < sy_full:
        print('error!')
  else:
        cnt = 0
        for yy in range(sy_full,sy):
            if npy.mod(yy,1972) == 0:
                mlenfeb = 29
                ylen = 366
            else:
                mlenfeb = 28
                ylen = 365
            cnt = cnt + ylen
  print('starting at '+str(cnt))

  yrange_plot =range(sy,ey+1)
  yrange =range(sy_full,ey_full+1)



  if (os.path.isfile(savename) == 1) & (plot_figs +compute_vals == 0):
    print('loading')
    fid = npy.load(savename)
    yrange = fid['yrange']
    si_area =  fid['si_area']
    myi_area = fid['myi_area'] 
    fyi_area = fid['fyi_area']
    yi_area = fid['yi_area']
    amb_area = fid['amb_area']
    owt_area = fid['owt_area']
    yax = fid['yax']
    m_points = fid['m_points']
    grid_area = fid['grid_area']
    lat=fid['lat']
    lon = fid['lon']
    WWS_area = fid['WWS_area']
    SWS_area = fid['SWS_area']
    Both_area = fid['Both_area']
    WWSnocbox = fid['WWSnocbox']
    SWSnocbox = fid['SWSnocbox']
    WWS_nocorner_area = fid['WWS_nocorner_area']
    SWS_nocorner_area = fid['SWS_nocorner_area']
    WWSbox = fid['WWSbox']
    SWSbox = fid['SWSbox']
    Bothbox = fid['Bothbox']
    landmask =fid['landmask']
    landmask_10 = fid['landmask_10']

    ## Quickly compute yax if not computing...
    yax = npy.nan*npy.ones([len(yrange)*366,1])
    for yr in yrange_plot:
        mst = 0
        if npy.mod(yr,1972) == 0:
            mlenfeb = 29
            ylen = 366
        else:
            mlenfeb = 28
            ylen = 365
        for m in range(1,13):
            mlen = 31
            if m in [4,6,9,11]:
                mlen = 30
            elif m == 2:
                mlen = mlenfeb
            for d in range(1,mlen+1):

                yax[cnt] = yr + (mst + d)/ylen

                cnt = cnt + 1

                print(str(yr)+'m'+str(m).zfill(2))
                mst = mst + mlen 
                #mcnt = mcnt + 1
  #### Finished computing yax
    
  else:

    plt.rcParams['axes.facecolor'] = 'white'
    if (plot_figs == 0) & (compute_vals == 0):
        ## we have not got a saved file! Turn compute_vals to 1
        compute_vals = 1

    si_area = npy.nan*npy.ones([len(yrange)*366,5])
    myi_area = npy.nan*npy.ones([len(yrange)*366,5])
    fyi_area = npy.nan*npy.ones([len(yrange)*366,5])
    yi_area = npy.nan*npy.ones([len(yrange)*366,5])
    amb_area = npy.nan*npy.ones([len(yrange)*366,5])
    owt_area = npy.nan*npy.ones([len(yrange)*366,5])

    yax = npy.nan*npy.ones([len(yrange)*366,1])
    m_points = npy.nan*npy.ones([len(yrange)*12,1])

    mcnt = 0

    grid_area = cell_area*1

    lat,lon,ice_conc,MYI,FYI,YI,OW,exist = get_conc_and_type(2019,12,30)

    WWS_area = npy.nansum(npy.nansum(grid_area*WWSbox*landmask,axis=1),axis=0)
    SWS_area = npy.nansum(npy.nansum(grid_area*SWSbox*landmask,axis=1),axis=0)
    Both_area = npy.nansum(npy.nansum(grid_area*Bothbox*landmask,axis=1),axis=0)

    WWSnocbox = WWSbox*1
    WWSnocbox[WWSbox+SWSbox==2] = 0
    SWSnocbox = SWSbox*1
    SWSnocbox[WWSbox+SWSbox==2] = 0

    WWS_nocorner_area = npy.nansum(npy.nansum(grid_area*WWSnocbox*landmask,axis=1),axis=0)
    SWS_nocorner_area = npy.nansum(npy.nansum(grid_area*SWSnocbox*landmask,axis=1),axis=0)

    for yr in yrange_plot:
        mst = 0
        if npy.mod(yr,1972) == 0:
            mlenfeb = 29
            ylen = 366
        else:
            mlenfeb = 28
            ylen = 365
        for m in range(1,13):
            mlen = 31
            if m in [4,6,9,11]:
                mlen = 30
            elif m == 2:
                mlen = mlenfeb
            for d in range(1,mlen+1):

                yax[cnt] = yr + (mst + d)/ylen

                lat,lon,ice_conc,MYI,FYI,YI,OW,exist = get_conc_and_type(yr,m,d)

                si_area[cnt,0] = npy.nansum(npy.nansum(ice_conc*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                si_area[cnt,1] = npy.nansum(npy.nansum(ice_conc*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                si_area[cnt,2] = npy.nansum(npy.nansum(ice_conc*grid_area*Bothbox,axis=1),axis=0)#/Both_area
                si_area[cnt,3] = npy.nansum(npy.nansum(ice_conc*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_nocorner_area
                si_area[cnt,4] = npy.nansum(npy.nansum(ice_conc*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_nocorner_area

                if exist == 1:
                    myivals = MYI*1#npy.zeros(npy.shape(typ)); #myivals[(typ==3.)] = 1;
                    fyivals = FYI*1
                    ambvals = 0;#npy.zeros(npy.shape(typ)); ambvals[typ==4] = 1                    
                    owtvals = OW*1;#npy.zeros(npy.shape(typ));  owtvals[typ==1] = 1                    
                    yivals = YI*1;#npy.zeros(npy.shape(typ));  owtvals[typ==1] = 1
                    #print(npy.nansum(myivals))
                    
                    if compute_vals:
                        myi_area[cnt,0] = npy.nansum(npy.nansum(myivals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                        myi_area[cnt,1] = npy.nansum(npy.nansum(myivals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                        myi_area[cnt,2] = npy.nansum(npy.nansum(myivals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                        myi_area[cnt,3] = npy.nansum(npy.nansum(myivals*grid_area*WWSnocbox,axis=1),axis=0)#/SWS_area
                        myi_area[cnt,4] = npy.nansum(npy.nansum(myivals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                        #print(npy.nansum(fyivals))
                        fyi_area[cnt,0] = npy.nansum(npy.nansum(fyivals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                        fyi_area[cnt,1] = npy.nansum(npy.nansum(fyivals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                        fyi_area[cnt,2] = npy.nansum(npy.nansum(fyivals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                        fyi_area[cnt,3] = npy.nansum(npy.nansum(fyivals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                        fyi_area[cnt,4] = npy.nansum(npy.nansum(fyivals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                        #print(npy.nansum(ambvals))
                        amb_area[cnt,0] = npy.nansum(npy.nansum(ambvals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                        amb_area[cnt,1] = npy.nansum(npy.nansum(ambvals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                        amb_area[cnt,2] = npy.nansum(npy.nansum(ambvals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                        amb_area[cnt,3] = npy.nansum(npy.nansum(ambvals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                        amb_area[cnt,4] = npy.nansum(npy.nansum(ambvals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                        #print(npy.nansum(owtvals))
                        owt_area[cnt,0] = npy.nansum(npy.nansum(owtvals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                        owt_area[cnt,1] = npy.nansum(npy.nansum(owtvals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                        owt_area[cnt,2] = npy.nansum(npy.nansum(owtvals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                        owt_area[cnt,3] = npy.nansum(npy.nansum(owtvals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                        owt_area[cnt,4] = npy.nansum(npy.nansum(owtvals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                        #print(npy.nansum(owtvals))
                        yi_area[cnt,0] = npy.nansum(npy.nansum(yivals*grid_area*WWSbox,axis=1),axis=0)#/WWS_area
                        yi_area[cnt,1] = npy.nansum(npy.nansum(yivals*grid_area*SWSbox,axis=1),axis=0)#/SWS_area
                        yi_area[cnt,2] = npy.nansum(npy.nansum(yivals*grid_area*Bothbox,axis=1),axis=0)#/SWS_area
                        yi_area[cnt,3] = npy.nansum(npy.nansum(yivals*grid_area*WWSnocbox,axis=1),axis=0)#/WWS_area
                        yi_area[cnt,4] = npy.nansum(npy.nansum(yivals*grid_area*SWSnocbox,axis=1),axis=0)#/SWS_area

                    plot_figs_now = plot_figs*1
                    savename_fig = figdir+'IceTypemaps/AllTypes/Alltypes_subplots_counter'+str(cnt).zfill(5)+'.png'
                    if only_when_not_exist:
                        if os.path.isfile(savename_fig):
                            plot_figs_now = 0
                    if plot_figs_now:
                        plt.figure()
                        plt.pcolormesh(npy.flipud(ice_conc*landmask),vmin=0,vmax=1,cmap='Blues_r'); plt.colorbar()
                        plt.contour(npy.flipud(WWSbox),colors='r')
                        plt.contour(npy.flipud(SWSbox),colors='g')
                        plt.contour(npy.flipud(Bothbox),colors='blue')
                        if use_OSISAF:
                            plt.contour(npy.flipud(status_flag),[100,101],colors='k')
                            plt.xlim([100,400])
                            plt.ylim([400,700])
                        else:
                            plt.contour(npy.flipud(landmask_10),[0.9,1],colors='k')
                            plt.xlim([50,320])
                            plt.ylim([300,570])
                        plt.title('SIC_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2))
                        plt.savefig(figdir+'SICmaps/SIC_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2)+'.png',bbox_inches='tight')
                        plt.close()
                        
                        fig,axs = plt.subplots(2,2,figsize=(10,10))
                        pl1 = 0; pl2 = 0
                        for icetype in ['MYI','FYI','YI','ice_conc']:
                            mcnt
                            thisf = eval(icetype)*1
                            #if icetype != 'OW':
                            #    thisf[OW==1] = -1
                            px=axs[pl1,pl2].contourf(npy.flipud(thisf*landmask),10,vmin=0,vmax=100,cmap='Spectral_r'); 
                            if pl1 + pl2 == 0:
                                ax = fig.add_axes([0.25,0.01,0.5,0.05])
                                plt.colorbar(px,cax=ax,orientation='horizontal')
                            #if icetype != 'OW':
                            #    axs[pl1,pl2].contourf(npy.flipud(OW),[100,101],colors='grey')
                            axs[pl1,pl2].contour(npy.flipud(WWSbox),colors='r')
                            axs[pl1,pl2].contour(npy.flipud(SWSbox),colors='g')
                            axs[pl1,pl2].contour(npy.flipud(Bothbox),colors='blue')
                            axs[pl1,pl2].contour(npy.flipud(MYI+FYI+YI),[15,16],colors='k')
                            if use_OSISAF:
                                axs[pl1,pl2].contour(npy.flipud(status_flag),[100,101],colors='k')
                                axs[pl1,pl2].set_xlim([100,400])
                                axs[pl1,pl2].set_ylim([400,700])
                            else:
                                plt.contour(npy.flipud(landmask_10),[0.9,1.1],colors='k')
                                plt.contour(npy.flipud(landmask_10),[0.5],colors='k')
                                axs[pl1,pl2].set_xlim([50,320])
                                axs[pl1,pl2].set_ylim([300,570])
                            axs[pl1,pl2].set_title(icetype+'_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2))
                            pl1 = pl1 + 1
                            if pl1 == 2:
                                pl1 = 0; pl2 = 1
                        plt.savefig(savename_fig,bbox_inches='tight')
                        #plt.savefig(figdir+'IceTypemaps/AllTypes/Alltypes_subplots_y'+str(yr)+'m'+str(m).zfill(2)+'d'+str(d).zfill(2)+'_counter'+str(cnt).zfill(5)+'.png',bbox_inches='tight')
                        plt.close()

                cnt = cnt + 1

            print(str(yr)+'m'+str(m).zfill(2))
            mst = mst + mlen 
            m_points[mcnt,0] = yr + mst/ylen
            mcnt = mcnt + 1
            
        if compute_vals:
          npy.savez(savename,
            yrange = yrange,
            si_area = si_area,
            myi_area = myi_area,
            fyi_area = fyi_area,
            yi_area = yi_area,
            amb_area = amb_area,
            owt_area = owt_area,
            yax = yax,
            m_points = m_points,
            grid_area = grid_area,
            lat=lat,
            lon = lon,
            WWS_area =WWS_area,
            SWS_area = SWS_area,
            Both_area = Both_area,
            WWSnocbox = WWSnocbox,
            SWSnocbox = SWSnocbox,
            WWS_nocorner_area = WWS_nocorner_area,
            SWS_nocorner_area = SWS_nocorner_area,
            WWSbox = WWSbox,
            SWSbox = SWSbox,
            Bothbox =Bothbox,
            landmask =landmask,
            landmask_10 = landmask_10)

print('done')

### Manually get time axis

In [ ]:
do_yax = 1
cnt = 0
if do_yax:
    yax = npy.nan*npy.ones([len(yrange)*366,1])
    for yr in yrange_plot:
        mst = 0
        if npy.mod(yr,1972) == 0:
            mlenfeb = 29
            ylen = 366
        else:
            mlenfeb = 28
            ylen = 365
        for m in range(1,13):
            mlen = 31
            if m in [4,6,9,11]:
                mlen = 30
            elif m == 2:
                mlen = mlenfeb
            for d in range(1,mlen+1):

                yax[cnt] = yr + (mst + d)/ylen
                print(mst+d)
                print(ylen)
                print(yax[cnt])
                print(yr+(mst)/ylen)

                cnt = cnt + 1

            print(str(yr)+'m'+str(m).zfill(2))
            mst = mst + mlen 

## Now do some plotting

### Movie if we want

In [ ]:
tes = 0
if tes == 1:
    import ffmpeg
    
    print(figdir)

    os.system("ffmpeg -r 1 -i /Users/heareg/Documents/Floes/working_directory/figures/Bremen/IceTypemaps/AllTypes/Alltypes_subplots_y*counter%01d.png -vcodec mpeg4 -y movie.mp4")
    print('done')
    for filename in os.listdir(figdir+'IceTypeMaps/AllTypes'):
        if (filename.endswith(".mp4")): #or .avi, .mpeg, whatever.
            os.system("ffmpeg -i {0} -f image2 -vf fps=fps=1 output%d.png".format(filename))
        else:
            continue



    def save_video():
        print('doing')
        os.system("ffmpeg -r 1 -i /Users/heareg/Documents/Floes/working_directory/figures/Bremen/IceTypemaps/AllTypes/Alltypes_subplots_y%01d.png -vcodec mpeg4 -y /Users/heareg/Documents/Floes/working_directory/figures/Bremen/IceTypemaps/AllTypes/movie.mp4")
        os.system("ffmpeg -f image2 -r 1/5 -i ./images/swissGenevaLake%01d.jpg -vcodec mpeg4 -y ./videos/swissGenevaLake.mp4")

        print('done')
        return

    save_video


### Bar chart

In [ ]:
def plot_barchart(regn,rarea,titstr,do_OW=1,do_leg=1):

    ### HERE IS SOME EDITING TO MAKE THE BAR CHART NICER
    sitot = si_area[:,regn]/rarea
    sitot[sitot>100] = npy.nan
    sitot[sitot<=0] = npy.nan
    sitot_next = sitot[1::]*1
    sitot_b4 = sitot[0:-1]*1
    sitot_b4[sitot_b4>10+sitot_next] = npy.nan
    sitot_next[sitot_next>10+sitot_b4] = npy.nan
    sitot_next[npy.isnan(sitot_b4)] = npy.nan
    sitot[1::] = sitot_next
    
    thisYI = yi_area[:,regn]/rarea
    thisYI[npy.isnan(sitot)] = 0
    #thisamb = amb_area[:,regn]/rarea
    thisamb = 100 - thisYI - myi_area[:,regn]/rarea - fyi_area[:,regn]/rarea-owt_area[:,regn]/rarea
    
    plt.figure(figsize=(15,4))
    plt.bar(yax[:,0],myi_area[:,regn]/rarea,width=ww,color='forestgreen',alpha=0.8,label='MYI')
    #plt.bar(yax[:,0],myi_area[:,regn]/rarea,width=ww,color='mediumvioletred',alpha=0.8)
    plt.bar(yax[:,0],fyi_area[:,regn]/rarea,bottom=myi_area[:,regn]/rarea,width=ww,color='gold',alpha=0.6,label='FYI')
    plt.bar(yax[:,0],thisYI,bottom=myi_area[:,regn]/rarea+fyi_area[:,regn]/rarea,
            width=ww,color='deepskyblue',alpha=0.6,label='YI')
    plt.bar(yax[:,0],thisamb,bottom=myi_area[:,regn]/rarea+fyi_area[:,regn]/rarea+thisYI,width=ww,
            color='grey',alpha=0.6,label='AMB')
    if do_OW == 1:
        plt.bar(yax[:,0],owt_area[:,regn]/rarea,bottom=myi_area[:,regn]/rarea+fyi_area[:,regn]/rarea+thisamb+thisYI,
            width=ww,color='blue',alpha=0.6, label='OW')
        if do_leg:
            plt.legend(ncol=2)
            #plt.legend(['MYI','FYI','YI','OW'],ncol=2)
    else:
        if do_leg:
            plt.legend(['MYI','FYI','AMB','YI'],ncol=2)

    plt.plot(yax[:,0],sitot,'k--',linewidth=2,label='all SI')
    #plt.plot(yax[:,0],si_area[:,regn]/rarea,'m--')
    yy = plt.gca()
    maxy = npy.nanmax([yy.get_ylim()])
    for nn in range(0,len(m_points)):
        plt.plot([m_points[nn],m_points[nn]],[0,maxy],'k--',linewidth=0.7)
    for nn in yrange:
        plt.plot([nn,nn],[0,maxy],'k',linewidth=1.5)
    plt.xlim([yrange[0]+0.05,yrange[-1]+0.35])
    plt.title(titstr,fontsize=14)
    plt.ylabel('% of ice type in domain',fontsize=12)
    plt.xlabel('year',fontsize=12)

    return

### Extra things still to be done:
### 1) smooth the daily ice type data to weekly?
### 2) limit x-axis to end of 2021 (as per rest of analysis)
### 3) check month letters on x-axis are not too cramped

In [ ]:
import matplotlib as mpl
cmapOSI = mpl.colors.ListedColormap(['blue','deepskyblue','gold','mediumvioletred','grey'])

if use_OSISAF:
    ww = 2*1./(len(yrange)*366)
else:
    ww = 4*1./(len(yrange)*366)
    
mlist = ['O','N','D','J','F','M','A','M','J','J','A','S','O','N','D','J','F','M','A','M','J','J','A','S','O','N','D','J','F','M','A','M','J','J','A','S','O','N','D','J','F','M','A']
mpos = npy.arange(2018+10/12-1.5/24,2022+4./12-1./24,1./12)

plot_barchart(1,SWS_area,'Southern Weddell Sea',do_leg=0)
fig = plt.gcf()
fig.patch.set_facecolor('xkcd:white')
plt.savefig(figdir+'SWS_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'.png',dpi=100,bbox_inches='tight')
plt.ylim([0,100])
ax = plt.gca()
for nn in range(0,len(mlist)):
    ax.annotate(mlist[nn],[mpos[nn],0.92],fontweight='bold',color='k',fontsize=12)
ax = plt.gca()
ax2 = ax.twinx()
ax2.plot(npy.arange(2018+10/12+1./24,2022+3./12+1./24,1./12),ATL20_thick_monthly_south_av[:,1],'r--',linewidth=3,label='SI freeboard')
#plt.legend(ncol=2)
ax2.set_ylabel('Sea ice freeboard (m)',fontsize=12)
ax2.yaxis.label.set_color('r')
ax2.spines["right"].set_edgecolor('r')
ax2.tick_params(axis='y', colors='r')
plt.savefig(figdir+'SWS_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'_withfreeboard.png',dpi=200,bbox_inches='tight')
ax2.set_ylim([0.1,0.6])
plt.savefig(figdir+'SWS_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'_withfreeboardlim.png',dpi=200,bbox_inches='tight')

plot_barchart(3,WWS_nocorner_area,'Western Weddell Sea')
plt.savefig(figdir+'WWS_no_corner_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'.png',dpi=200,bbox_inches='tight')
plt.ylim([0,100])
ax = plt.gca()
for nn in range(0,len(mlist)):
    ax.annotate(mlist[nn],[mpos[nn],0.92],fontweight='bold',color='k',fontsize=12)
ax2 = ax.twinx()
plt.plot(npy.arange(2018+10/12+1./24,2022+3./12+1./24,1./12),ATL20_thick_monthly_west_av[:,1],'r--',linewidth=3,label='SIT')
#ax[0].set_xticks(npy.cumsum(mstarts))
#ax[0].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul'])
ax2.annotate('hello',[2019.5,20])
ax2.set_ylabel('Sea ice freeboard (m)',fontsize=12)
ax2.set_ylabel('Sea ice freeboard (m)',fontsize=12)
ax2.yaxis.label.set_color('r')
ax2.spines["right"].set_edgecolor('r')
ax2.tick_params(axis='y', colors='r')
fig = plt.gcf()
fig.patch.set_facecolor('xkcd:white')
plt.savefig(figdir+'WWS_no_corner_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'_withfreeboard.png',dpi=200,bbox_inches='tight')
ax2.set_ylim([0.1,0.6])
plt.savefig(figdir+'WWS_no_corner_box_stacked_bar_proportion_of_100_'+str(sy)+'-'+str(ey)+'_withfreeboardlim.png',dpi=200,bbox_inches='tight')
print(figdir)

print('---- AREA OF EACH REGION ----')
print('South area is '+str(SWS_area)+' km2')
print('West area is '+str(WWS_nocorner_area)+' km2')
print('---- THESE SHOULD BE CHECKED - THEY"RE VERY DIFFERENT!! ----')

### Plot scatterplots with lag

In [ ]:
Fbd_y = npy.arange(2018+10/12+1./24,2022+3./12+1./24,1./12)

Wind = 3
Sind = 1

SI_W_S = npy.zeros([len(Fbd_y),2])
MYI_W_S = npy.zeros([len(Fbd_y),2])
FYI_W_S = npy.zeros([len(Fbd_y),2])
YI_W_S = npy.zeros([len(Fbd_y),2])
OW_W_S = npy.zeros([len(Fbd_y),2])

for mcnt in range(1,len(Fbd_y)):
    
    ind = npy.nonzero((yax>=Fbd_y[mcnt])&(yax<Fbd_y[mcnt]+1./12))
    SI_W_S[mcnt,0] = npy.nanmean(si_area[ind,3])
    SI_W_S[mcnt,1] = npy.nanmean(si_area[ind,1])
    MYI_W_S[mcnt,0] = npy.nanmean(myi_area[ind,3])
    MYI_W_S[mcnt,1] = npy.nanmean(myi_area[ind,1])
    FYI_W_S[mcnt,0] = npy.nanmean(fyi_area[ind,3])
    FYI_W_S[mcnt,1] = npy.nanmean(fyi_area[ind,1])
    YI_W_S[mcnt,0] = npy.nanmean(yi_area[ind,3])
    YI_W_S[mcnt,1] = npy.nanmean(yi_area[ind,1])
    OW_W_S[mcnt,0] = npy.nanmean(owt_area[ind,3])
    OW_W_S[mcnt,1] = npy.nanmean(owt_area[ind,1])


lagvals = [-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6]
for itype in ['SI','MYI','FYI']:  
  for lagval in lagvals:
    
    fig, (P1,P2) = plt.subplots(1,2,figsize=(10,5))
        
    if lagval > 0:
        fvals = ATL20_thick_monthly_west_av[lagval::,1]
        sivals = eval(itype+'_W_S')[0:len(SI_W_S)-lagval,0]/1e6
        P1.scatter(fvals,sivals,c=Fbd_y[lagval::],cmap='jet')
        #P1.set_ylim([40,70])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P1.set_title(itype+' West, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P1.grid()
        
        fvals = ATL20_thick_monthly_south_av[lagval::,1]
        sivals = eval(itype+'_W_S')[0:len(SI_W_S)-lagval,1]/1e6
        P2.scatter(fvals,sivals,c=Fbd_y[lagval::],cmap='jet')
        #P2.set_ylim([38,46])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P2.set_title(itype+' South, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P2.grid()
    elif lagval == 0:
        fvals = ATL20_thick_monthly_west_av[:,1]
        sivals = eval(itype+'_W_S')[:,0]/1e6
        P1.scatter(fvals,sivals,c=Fbd_y[lagval::],cmap='jet')
        #P1.set_ylim([40,70])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P1.set_title(itype+' West, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P1.grid()
        
        fvals = ATL20_thick_monthly_south_av[:,1]
        sivals = eval(itype+'_W_S')[:,1]/1e6
        P2.scatter(fvals,sivals,c=Fbd_y[lagval::],cmap='jet')
        #P2.set_ylim([38,46])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P2.set_title(itype+' South, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P2.grid()       
    else:
        fvals = ATL20_thick_monthly_west_av[0:len(SI_W_S)-npy.abs(lagval),1]
        sivals = eval(itype+'_W_S')[npy.abs(lagval)::,0]/1e6
        P1.scatter(fvals,sivals,c=Fbd_y[npy.abs(lagval)::],cmap='jet')
        #P1.set_ylim([40,70])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P1.set_title(itype+' West, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P1.grid()
        
        fvals = ATL20_thick_monthly_south_av[0:len(SI_W_S)-npy.abs(lagval),1]
        sivals = eval(itype+'_W_S')[npy.abs(lagval)::,1]/1e6
        P2.scatter(fvals,sivals,c=Fbd_y[npy.abs(lagval)::],cmap='jet')
        #P2.set_ylim([38,46])
        ind = npy.where((fvals>0)&(sivals>0))
        corrval = npy.corrcoef(fvals[ind],sivals[ind])
        P2.set_title(itype+' South, lagval = '+str(lagval)+', corr '+str(npy.round(corrval[0,1],2)))
        P2.grid()
    cax = fig.add_axes([0.96,0.1,0.06,0.8])
    cax.pcolormesh([0,1],Fbd_y,npy.tile(Fbd_y.T,[2,1]).T,cmap='jet')
    #cax.set_ylim([Fbd_y[0],Fbd_y[-1]])
    #cax.set_yticks(npy.arange(0,len(Fbd_y),4))
    #cax.set_yticklabels(npy.round(100*Fbd_y[0:-1:4])/100)
    cax.set_xticklabels([])

fig, (P1,P2) = plt.subplots(1,2,figsize=(10,5))
for nn in npy.arange(0,len(SI_W_S),12):
    P1.scatter(ATL20_thick_monthly_west_av[nn:nn+12,1],SI_W_S[nn:nn+12,0]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P1.set_ylim([40,70])
P1.grid()
ind = npy.where((ATL20_thick_monthly_west_av[:,1]>0)&(SI_W_S[:,0]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_west_av[ind,1],SI_W_S[ind,0]/1e6)
P1.set_title('West freeboard and SI: '+str(npy.round(corrval[0,1],2)))
for nn in npy.arange(0,len(SI_W_S),12):
    P2.scatter(ATL20_thick_monthly_south_av[nn:nn+12,1],SI_W_S[nn:nn+12,1]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P2.set_ylim([38,46])
P2.grid()
ind = npy.where((ATL20_thick_monthly_south_av[:,1]>0)&(SI_W_S[:,1]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_south_av[ind,1],SI_W_S[ind,1]/1e6)
P2.set_title('South freeboard and SI: '+str(npy.round(corrval[0,1],2)))
cax = fig.add_axes([0.94,0.1,0.06,0.8])
cax.pcolormesh(npy.tile(npy.arange(1,13).T,[2,1]).T,cmap='hsv')
cax.set_yticks(npy.arange(1,13))
cax.set_yticklabels(mlist[0:12])
cax.set_xticklabels([])

fig, (P1,P2) = plt.subplots(1,2,figsize=(10,5))
for nn in npy.arange(0,len(SI_W_S),12):
    P1.scatter(ATL20_thick_monthly_west_av[nn:nn+12,1],MYI_W_S[nn:nn+12,0]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P1.set_ylim([15,40])
P1.grid()
ind = npy.where((ATL20_thick_monthly_west_av[:,1]>0)&(MYI_W_S[:,0]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_west_av[ind,1],MYI_W_S[ind,0]/1e6)
P1.set_title('West freeboard and MYI: '+str(npy.round(corrval[0,1],2)))
for nn in npy.arange(0,len(SI_W_S),12):
    P2.scatter(ATL20_thick_monthly_south_av[nn:nn+12,1],MYI_W_S[nn:nn+12,1]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P2.set_ylim([10,20])
P2.grid()
ind = npy.where((ATL20_thick_monthly_south_av[:,1]>0)&(MYI_W_S[:,1]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_south_av[ind,1],MYI_W_S[ind,1]/1e6)
P2.set_title('South freeboard and MYI: '+str(npy.round(corrval[0,1],2)))
cax = fig.add_axes([0.94,0.1,0.06,0.8])
cax.pcolormesh(npy.tile(npy.arange(1,13).T,[2,1]).T,cmap='hsv')
cax.set_yticks(npy.arange(1,13))
cax.set_yticklabels(mlist[0:12])
cax.set_xticklabels([])

fig, (P1,P2) = plt.subplots(1,2,figsize=(10,5))
for nn in npy.arange(0,len(SI_W_S),12):
    P1.scatter(ATL20_thick_monthly_west_av[nn:nn+12,1],FYI_W_S[nn:nn+12,0]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P1.set_ylim([10,35])
P1.grid()
ind = npy.where((ATL20_thick_monthly_west_av[:,1]>0)&(FYI_W_S[:,0]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_west_av[ind,1],FYI_W_S[ind,0]/1e6)
P1.set_title('West freeboard and FYI: '+str(npy.round(corrval[0,1],2)))
for nn in npy.arange(0,len(SI_W_S),12):
    P2.scatter(ATL20_thick_monthly_south_av[nn:nn+12,1],FYI_W_S[nn:nn+12,1]/1e6,c=Fbd_y[nn:nn+12],cmap='hsv')
P2.set_ylim([10,35])
P2.grid()
ind = npy.where((ATL20_thick_monthly_south_av[:,1]>0)&(FYI_W_S[:,1]>0))
corrval = npy.corrcoef(ATL20_thick_monthly_south_av[ind,1],FYI_W_S[ind,1]/1e6)
P2.set_title('South freeboard and FYI: '+str(npy.round(corrval[0,1],2)))
cax = fig.add_axes([0.94,0.1,0.06,0.8])
cax.pcolormesh(npy.tile(npy.arange(1,13).T,[2,1]).T,cmap='hsv')
cax.set_yticks(npy.arange(1,13))
cax.set_yticklabels(mlist[0:12])
cax.set_xticklabels([])